In [5]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from toydown import ToyDown
from tqdm import tqdm, trange





# for district_type, ds in [("tract_recom", tract_recom)]:
#     print("Starting {} calculations".format(district_type), flush=True)
# #     print(ds[0])
#     var = toy.assign_district_tree_variance(ds[0])
#     print(var)
#     break

In [9]:
filename = "Blocks/dallas_county_blocks10.shp"
toy = ToyDown(filename, "GEOID10", "TOTPOP10")

eps_budget = 0.2                  

epsilon_split = {"equal": [0.2,0.2,0.2,0.2,0.2], 
                 "top_heavy": [1/2, 1/4, 1/12, 1/12, 1/12],
                 "mid_heavy": [1/12, 1/6, 1/2, 1/6, 1/12], 
                 "bottom_heavy": [1/12, 1/12, 1/12, 1/4, 1/2],
                 "bg_heavy": [1/12, 1/6, 1/6, 1/2, 1/12]}

toy.set_noising_params(eps_budget, epsilon_split["equal"])
toy.noise_and_adjust()

SyntaxError: invalid syntax (<ipython-input-9-8e93e8001fda>, line 12)

In [3]:
def district_frag_scores(h, districts):
    num_districts = len(districts)
    frag_scores = np.zeros(num_districts)
    for i in tqdm(range(num_districts)):
        frag_scores[i] = h.assign_district_tree_variance(districts[i], eps=1, 
                                                         eps_splits=[np.sqrt(2)]*5,
                                                         sensitivity=1)
    return frag_scores

tract_recom = np.load("100_recom_tract_parts_comp.npy", allow_pickle=True)[0]["dicts"]

In [6]:
for district_type, ds in [("tract_recom", tract_recom)]:
    print("Starting {} calculations".format(district_type), flush=True)
    frag_scores = district_frag_scores(toy, ds)

Starting tract_recom calculations


  0%|          | 0/399 [00:00<?, ?it/s]

Node(tag=48, identifier=48, data=GeoUnit(name=48, unnoised_pop=2368139, parent=None, identifier=48, level=0, noised_pop=2368156.7567961817, noise=17.756796181800283, noise_type=laplacian, adjusted_pop=2368156.7567961817, error=17.756796181667596))


  0%|          | 1/399 [00:00<03:14,  2.05it/s]

Node(tag=48, identifier=48, data=GeoUnit(name=48, unnoised_pop=2368139, parent=None, identifier=48, level=0, noised_pop=2368156.7567961817, noise=17.756796181800283, noise_type=laplacian, adjusted_pop=2368156.7567961817, error=17.756796181667596, weight=0.2533081285444234))


...

 99%|█████████▉| 397/399 [02:26<00:00,  2.73it/s]

Node(tag=48, identifier=48, data=GeoUnit(name=48, unnoised_pop=2368139, parent=None, identifier=48, level=0, noised_pop=2368156.7567961817, noise=17.756796181800283, noise_type=laplacian, adjusted_pop=2368156.7567961817, error=17.756796181667596, weight=0.24007561436672967))


100%|█████████▉| 398/399 [02:27<00:00,  2.72it/s]

Node(tag=48, identifier=48, data=GeoUnit(name=48, unnoised_pop=2368139, parent=None, identifier=48, level=0, noised_pop=2368156.7567961817, noise=17.756796181800283, noise_type=laplacian, adjusted_pop=2368156.7567961817, error=17.756796181667596, weight=0.2381852551984877))


100%|██████████| 399/399 [02:27<00:00,  2.70it/s]


In [7]:
frag_scores

array([100.12087578, 107.54732151, 100.61333043,  92.28122755,
       100.61333043,  84.80007576,  95.51897327,  92.83028577,
       105.33033044, 110.49261902,  85.3981904 ,  92.28122755,
        97.60199899, 106.22844758, 103.95485293, 107.54732151,
        90.61141148,  91.72839577, 110.08318295, 112.0926276 ,
        91.72839577, 103.48881329, 104.41711901,  85.99253147,
        91.17179041,  96.56803328, 108.40770295, 107.54732151,
        98.11332149, 107.97939902,  83.59252576, 107.11147044,
        96.04539006,  93.37557041,  99.62464757,  82.9830904 ,
        94.45481899, 109.66997331, 107.97939902,  93.37557041,
        94.98878292,  97.08690292, 106.22844758,  82.9830904 ,
        90.61141148,  88.90763326,  98.11332149,  90.04725898,
       105.7812758 ,  94.98878292,  90.04725898,  93.91708149,
       107.54732151,  91.17179041, 107.11147044,  87.75291326,
       105.7812758 , 107.54732151, 109.25299009, 113.63225903,
        90.04725898, 104.41711901,  94.98878292,  85.39

In [8]:
def analytical_district_variances(h, districts, num_attributes=1):
    vrs = {}
    num_districts = len(districts)
    for split in ["equal", "top_heavy", "mid_heavy", "bottom_heavy", "bg_heavy"]:
        print(split)
        vs = np.array([h.assign_district_tree_variance(dist, eps=1, 
                         eps_splits=epsilon_split[split]) for dist in districts])*num_attributes
        vrs[split] = vs
    return vrs

In [ ]:
"""
hierarchy = [("Country", 1, 810.), 
             ("State", 3, 270.), 
             ("County", 3, 90.), 
             ("Dist", 3, 30.)]

eps_budget = 0.2                  
eps_splits = [0.25, 0.25, 0.25, 0.25] 

toy = ToyDown.from_hierarchy(hierarchy, eps_budget, eps_splits)
toy.show() 

# A demonstration of Proposition 1, calculated for CountryState1
branching = 3
exp_1 = toy.get_node("CountryState1").data.noise
exp_2 = ( toy.get_node("CountryState1").data.noise \
        + toy.get_node("CountryState2").data.noise \
        + toy.get_node("CountryState3").data.noise ) / branching
exp_3 = toy.get_node("Country").data.error / branching

math.isclose(exp_1 - exp_2 + exp_3,
             toy.get_node("CountryState1").data.error,
             abs_tol=0.00001)

# 6 -level Trees
hierarchy = [("Country", 1, 900000.), 
             ("State", 3, 300000.), 
             ("County", 3, 100000.), 
             ("Tract", 10, 10000.),
             ("BlockGroup", 10, 1000.),
             ("Block", 10, 100.)]

eps_budget = 0.4998 # I picked a random number from one of Peter's runs
eps_splits = [0, 0, 0.25, 0.25, 0.25, 0.25] 

toy = ToyModel(hierarchy, eps_budget, eps_splits)
toy.noise_and_adjust()
toy.show() 

# First, a sanity check that the equation at the end of Prop 4 works
# We will do this check on CountryState1County1Tract1
p = 0.5
dist_blocks = []
other_blocks = []
tract = toy.get_node("CountryState1County1Tract1")
block_groups = toy.children(tract.identifier)

def set_diff(a, b):
    return list(set(a).difference(set(b)))

for bg in block_groups:
    blocks = toy.children(bg.identifier)
    chosen_blocks = np.random.choice(blocks, int(p*10), replace=False)
    
    dist_blocks.extend(chosen_blocks)
    other_blocks.extend(set_diff(blocks, chosen_blocks))

dist_blk_noises = [blk.data.noise for blk in dist_blocks]
other_blk_noises = [blk.data.noise for blk in other_blocks]
blk_errors = [blk.data.error for blk in dist_blocks]

math.isclose((1-p) * sum(dist_blk_noises) - p * sum(other_blk_noises) + p * tract.data.error,
             sum(blk_errors),
             abs_tol = 0.000001)

# get blk errors for p from 0 to 1
def get_district_errors_at_p(p, tract_name):
    dist_blocks = []
    other_blocks = []
    tract = toy.get_node(tract_name)
    block_groups = toy.children(tract.identifier)

    for bg in block_groups:
        blocks = toy.children(bg.identifier)
        chosen_blocks = np.random.choice(blocks, int(p*10), replace=False)
        dist_blocks.extend(chosen_blocks)

    raw_errors = [blk.data.error for blk in dist_blocks]
    abs_errors = [abs(err) for err in raw_errors]
    
    return sum(raw_errors), sum(abs_errors)

raw_error_sums = []
abs_error_sums = []
ps = np.linspace(0,1,11)
for p in ps:
    raw_err, abs_err = get_district_errors_at_p(p, "CountryState1County1Tract1")
    raw_error_sums.append(raw_err)
    abs_error_sums.append(abs_err)
    
plt.plot(ps, raw_error_sums) 

# we are probably more interested in the absolute value of the errors than the actual sum of errors
plt.plot(ps, abs_error_sums)
"""